In [2]:
from ortools.linear_solver import pywraplp

In [3]:
def load_data(path = "12nodes"):
    with open(path, 'r') as f:
        inputData = f.readlines()
    
    N = int(inputData[0].strip())
    node_list=[]
    for node in inputData[1:]:
        #del '\n' 
        node = node.strip()
        #split by ' '
        node = node.split(' ')
        node_list.append((int(node[0]), int(node[1])))

    return node_list, N

node_list, N = load_data()

In [4]:
import numpy as np

def DistanceMatrix(cities, n):
    dis_matrix = np.zeros([n,n])
    
    for i in range(n):
        for j in range(i+1, n):
            a = np.array(cities[i])
            b = np.array(cities[j])
            c = a - b

            dis_matrix[i, j] = np.sqrt(np.sum(c*c))
            dis_matrix[j, i] = dis_matrix[i, j]
    
    return np.around(dis_matrix, 2)

c = DistanceMatrix(node_list, N)

K = 2

In [14]:
# Miller-Tucker-Zemlin
solver = pywraplp.Solver.CreateSolver('SCIP')

rate = 0.5

infinity = solver.infinity()

X = []
Y = []

max_dis = solver.NumVar(0, infinity, 'max_dis')
# total_dis = solver.NumVar(0, infinity, 'total_dis')

for k in range(K):
    X_k = []
    Y_k = []
    for i in range(N):
        X_i = []
        y_i = solver.IntVar(0, N, 'y({}, {})'.format(k, i))
        Y_k.append(y_i)
        for j in range(N):
            x = solver.IntVar(0, 1, 'x({},{}, {})'.format(k, i, j))
            X_i.append(x)
        X_k.append(X_i)

    X.append(X_k)
    Y.append(Y_k)

for k in range(K):
    for i in range(1, N):
        s_i_j = 0
        s_j_i = 0
        for j in range(N):
            if i != j:
                s_i_j += X[k][i][j]
                s_j_i += X[k][j][i]

                solver.Add(Y[k][j] - Y[k][i] + N*(1 - X[k][i][j]) >= 1)


        solver.Add(s_i_j - s_j_i == 0)


for i in range(1, N):
    s_i_j = 0
    s_j_i = 0
    for j in range(N):
        for k in range(K):
            if i != j:
                s_i_j += X[k][i][j]
                s_j_i += X[k][j][i]

    solver.Add(s_i_j == 1)
    solver.Add(s_j_i == 1)



for k in range(K):
    s_0_i = 0
    s_i_0 = 0

    for i in range(1, N):
        s_0_i += X[k][0][i]
        s_i_0 += X[k][i][0]

    solver.Add(s_0_i == 1)
    solver.Add(s_i_0 == 1)


ct_max_dis = []
for k in range(K):
    ct = solver.Constraint(-infinity, 0, 'ct')
    for i in range(N):
        for j in range(N):
            ct.SetCoefficient(X[k][i][j], c[i][j])

    ct.SetCoefficient(max_dis, -1)
    ct_max_dis.append(ct)



objective = solver.Objective()
for k in range(K):
    for i in range(N):
        for j in range(N):
            objective.SetCoefficient(X[k][i][j], 1*c[i][j])


objective.SetCoefficient(max_dis, 1)

objective.SetMinimization()


status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())

else:
    print('The problem does not have an optimal solution.')

print('\nNumber of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())


Solution:
Objective value = 209.56000000000003

Number of variables = 313
Number of constraints = 292

Advanced usage:
Problem solved in 7837.000000 milliseconds
Problem solved in 11946 iterations
Problem solved in 383 branch-and-bound nodes


In [18]:
# Add SEC

from tabnanny import check


solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()

max_dis = solver.NumVar(0, infinity, 'max_dis')
# total_dis = solver.NumVar(0, infinity, 'total_dis')
X = []

for k in range(K):
    X_k = []
    for i in range(N):
        X_i = []
        for j in range(N):
            x = solver.IntVar(0, 1, 'x({},{}, {})'.format(k, i, j))
            X_i.append(x)
        X_k.append(X_i)

    X.append(X_k)



for k in range(K):
    for i in range(1, N):
        s_i_j = 0
        s_j_i = 0
        for j in range(N):
            if i != j:
                s_i_j += X[k][i][j]
                s_j_i += X[k][j][i]

        solver.Add(s_i_j - s_j_i == 0)


for i in range(1, N):
    s_i_j = 0
    s_j_i = 0
    for j in range(N):
        for k in range(K):
            if i != j:
                s_i_j += X[k][i][j]
                s_j_i += X[k][j][i]

    solver.Add(s_i_j == 1)
    solver.Add(s_j_i == 1)



for k in range(K):
    s_0_i = 0
    s_i_0 = 0

    for i in range(1, N):
        s_0_i += X[k][0][i]
        s_i_0 += X[k][i][0]

    solver.Add(s_0_i == 1)
    solver.Add(s_i_0 == 1)
    


ct_max_dis = []
for k in range(K):
    ct = solver.Constraint(-infinity, 0, 'ct')
    for i in range(N):
        for j in range(N):
            ct.SetCoefficient(X[k][i][j], c[i][j])

    ct.SetCoefficient(max_dis, -1)
    ct_max_dis.append(ct)



objective = solver.Objective()
for k in range(K):
    for i in range(N):
        for j in range(N):
            objective.SetCoefficient(X[k][i][j], 1*c[i][j])


objective.SetCoefficient(max_dis, 1)

objective.SetMinimization()

# status = solver.Solve()

# while(True):
for t in range(100):
    status = solver.Solve()

    check = 0
    
    if status == pywraplp.Solver.OPTIMAL:
        #check
        for k in range(K):
            mark = [0 for i in range(N)]
            curr = 0
            mark[0] = 1
            S = [0]
            for j in range(N-1):
                for i in range(N):
                    if X[k][curr][i].solution_value() == 1:
                        curr = i
                        break

                if mark[curr] == 0:
                    mark[curr] = 1
                    S.append(curr)
                else:
                    break
            
            num = 0
            mark = [0 for i in range(N)]
            for i in range(N):
                for j in range(N):
                    if X[k][i][j].solution_value() == 1:
                        mark[i] = 1
                        mark[j] = 1

            for i in range(N):
                num += mark[i]

            if len(S) == num:
                check += 1
                continue

            total = 0
            for i in S:
                for j in S:
                    if i != j:
                        total += X[k][i][j]

            solver.Add(total <= len(S) - 1)

        if check == K:
            break
                    
    else:
        print('The problem does not have an optimal solution.')
        break


print('Solution:')
print('Objective value =', solver.Objective().Value())
print('\nNumber of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())


Objective value = 167.07999999999998
Objective value = 167.07999999999998
Objective value = 170.38999999999996
Objective value = 173.57
Objective value = 173.87999900000034
Objective value = 174.07
Objective value = 175.23000000000002
Objective value = 175.4099999999999
Objective value = 176.24
Objective value = 178.41999999999996
Objective value = 178.64999999999998
Objective value = 179.22000000000003
Objective value = 179.22
Objective value = 181.17999999999998
Objective value = 182.87
Objective value = 183.13
Objective value = 183.28
Objective value = 186.12999999999994
Objective value = 186.62
Objective value = 186.86
Objective value = 187.29
Objective value = 187.57999999999993
Objective value = 188.86999999999995
Objective value = 189.89999999999998
Objective value = 190.63999999999996
Objective value = 191.17999999999998
Objective value = 192.45000000000005
Objective value = 192.62
Objective value = 192.62000000000077
Objective value = 192.72000000000008
Objective value = 192.7